In [ ]:
from __future__ import print_function

import numpy as np
from numpy import genfromtxt

import keras
from keras.models import Model
from keras.layers import Input, Dense, Merge, Dropout

In [ ]:
# import set from csv
test_data = genfromtxt(fname='chord_data.csv', delimiter=',')
test_data.shape

# split labels, train and test set
labels = test_data[:,0:2]
split_point = np.ceil((len(test_data)*0.7)).astype(int)
train_set = test_data[0:split_point,2:]
test_set = test_data[split_point:,2:]
train_labels = labels[0:split_point,0]
test_labels = labels[split_point:,0]
print('Train set shape:', train_set.shape)
print('Test set shape:', test_set.shape)

# convert to binary class matrices
# (root classification)
targets = 12

train_labels = keras.utils.to_categorical(train_labels, targets)
test_labels = keras.utils.to_categorical(test_labels, targets)
print('Train labels shape:', train_labels.shape)
print('Test labels shape:', test_labels.shape)

In [ ]:
# define the model for chord root classification
batch_size = 64
epochs = 10

def get_root_model():   
    a = Input(shape=(12,))
    b = Dense(128, activation='relu')(a)
    b = Dropout(0.5)(b)
    c = Dense(256, activation='relu')(b)  
    c = Dropout(0.5)(c)
    out = Dense(targets, activation='softmax')(c)        
        
    return Model(inputs=a, outputs=out)

In [ ]:
# Root model configuration
root_model = get_root_model()
root_model.compile(loss='categorical_crossentropy',
            optimizer='sgd',
            metrics=['accuracy'])

root_model.fit(train_set, train_labels,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(test_set, test_labels),
              verbose=0,
              shuffle=True
              )

root_model.summary()

In [ ]:
score = root_model.evaluate(test_set, test_labels, verbose=0)
print('Test accuracy:', score[1])

In [ ]:
# define the model for type classification (major, minor)

def get_type_model():   
    a = Input(shape=(12,))
    b = Dense(128, activation='tanh')(a)
    c = Dense(256, activation='tanh')(b)  
    c = Dropout(0.2)(c)
    out = Dense(1, activation='sigmoid')(c)        
        
    return Model(inputs=a, outputs=out)

In [ ]:
# Type model configuration
type_model = get_type_model()
type_model.compile(loss='binary_crossentropy',
            optimizer='adadelta',
            metrics=['accuracy'])

# change label so that they represent the type
train_labels = labels[0:split_point,1]
test_labels = labels[split_point:,1]

type_model.fit(train_set, train_labels,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(test_set, test_labels),
              verbose=0,
              shuffle=True
              )


In [ ]:
score = type_model.evaluate(test_set, test_labels, verbose=0)
print('Test accuracy:', score[1])

In [ ]:
full_labels = labels[split_point:,:].astype(int)
estimate_full_labels = np.zeros(full_labels.shape, dtype=int)
classification = np.zeros((len(full_labels),1), dtype=bool)

# Chord estimation
root_estimates = root_model.predict(test_set)
type_estimates = type_model.predict(test_set)

In [ ]:
# Comparison between estimates and true labels
for n in range(len(test_set)):
    estimate_full_labels[n,0] = np.argmax(root_estimates[n])
    estimate_full_labels[n,1] = np.round(type_estimates[n])
    # compare the labels and set the classification matrix True for the index
    if (estimate_full_labels[n]==full_labels[n]).all():
        classification[n] = True

In [ ]:
acc = np.sum(classification)/len(test_set)
print('Accuracy:', acc)